In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import psycopg2

In [2]:
VG_file = "Video_game_scores.csv"
VG_df = pd.read_csv(VG_file)
VG_df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
VG_sales = "vgsales.csv.zip"
VG_sales_df = pd.read_csv(VG_sales)
VG_sales_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
# Defining columns of interest from imported data
count_cols = ["Name", "Platform", "Publisher", "Critic_Score", "User_Score"]

# Dropping columns outside of defined count_cols
score_transformed = VG_df[count_cols].copy()
score_transformed = score_transformed.rename(columns={"Critic_Score": "critic_score",
                                                      "Publisher": "Console", 
                                                      "User_Score": "user_score"})

# Filling any NA values to avoid pandas conflicts
score_transformed = score_transformed.replace(0, np.nan)
score_transformed = score_transformed.dropna(how='all', axis=0)
score_transformed = score_transformed.replace(np.nan, 0)

score_transformed = score_transformed[score_transformed['critic_score'] !=0]
score_transformed = score_transformed[score_transformed['user_score'] !=0]
score_transformed = score_transformed[score_transformed['Console'] !=0]


# Verifying dataframe columns were transformed correctly
score_transformed.head()

,Name,Platform,Console,critic_score,user_score
0,Wii Sports,Wii,Nintendo,76.0,8
2,Mario Kart Wii,Wii,Nintendo,82.0,8.3
3,Wii Sports Resort,Wii,Nintendo,80.0,8
6,New Super Mario Bros.,DS,Nintendo,89.0,8.5
7,Wii Play,Wii,Nintendo,58.0,6.6


In [5]:
sc = score_transformed.groupby('Console')
sc.mean()

,critic_score
Console,
10TACLE Studios,42.000000
1C Company,73.000000
2D Boy,90.000000
3DO,57.470588
505 Games,61.926316
...,...
Zoo Games,47.000000
Zushi Games,53.500000
bitComposer Games,62.500000


In [13]:
VG_df.count()

Name               16717
Platform           16719
Year_of_Release    16450
Genre              16717
Publisher          16665
NA_Sales           16719
EU_Sales           16719
JP_Sales           16719
Other_Sales        16719
Global_Sales       16719
Critic_Score        8137
Critic_Count        8137
User_Score         10015
User_Count          7590
Developer          10096
Rating              9950
dtype: int64

In [14]:
score_transformed.count()

Name            8095
Platform        8095
Console         8095
critic_score    8095
user_score      8095
dtype: int64

In [6]:
score_transformed.sort_values(by='critic_score', ascending=False)
score_transformed.groupby('Console')

In [7]:
# Defining columns of interest from imported data
count_cols = ["Name", "Platform", "Publisher", "NA_Sales", "EU_Sales", "JP_Sales", "Global_Sales" ]

# Dropping columns outside of defined count_cols
sales_transformed = VG_sales_df[count_cols].copy()
sales_transformed = sales_transformed.rename(columns={"Publisher": "Console",
                                                      "Platform": "platform"})

# Filling any NA values to avoid pandas conflicts
sales_transformed = sales_transformed.replace(0, np.nan)
sales_transformed = sales_transformed.dropna(how='all', axis=0)
sales_transformed = sales_transformed.replace(np.nan, 0)


# Verifying dataframe columns were transformed correctly
sales_transformed.head()

,Name,platform,Console,NA_Sales,EU_Sales,JP_Sales,Global_Sales
0,Wii Sports,Wii,Nintendo,41.49,29.02,3.77,82.74
1,Super Mario Bros.,NES,Nintendo,29.08,3.58,6.81,40.24
2,Mario Kart Wii,Wii,Nintendo,15.85,12.88,3.79,35.82
3,Wii Sports Resort,Wii,Nintendo,15.75,11.01,3.28,33.00
4,Pokemon Red/Pokemon Blue,GB,Nintendo,11.27,8.89,10.22,31.37


In [8]:
sales_transformed.sort_values(by='Global_Sales', ascending=False)

,Name,platform,Console,NA_Sales,EU_Sales,JP_Sales,Global_Sales
0,Wii Sports,Wii,Nintendo,41.49,29.02,3.77,82.74
1,Super Mario Bros.,NES,Nintendo,29.08,3.58,6.81,40.24
2,Mario Kart Wii,Wii,Nintendo,15.85,12.88,3.79,35.82
3,Wii Sports Resort,Wii,Nintendo,15.75,11.01,3.28,33.00
4,Pokemon Red/Pokemon Blue,GB,Nintendo,11.27,8.89,10.22,31.37
...,...,...,...,...,...,...,...
16186,BattleForge,PC,Electronic Arts,0.00,0.01,0.00,0.01
16187,Jewel Quest II,PC,Avanquest,0.00,0.01,0.00,0.01
16188,Toro to Morimori,PS3,Sony Computer Entertainment,0.00,0.00,0.01,0.01
16189,Sonic & All-Stars Racing Transformed,PC,Sega,0.00,0.01,0.00,0.01


In [9]:
connection_string = "postgres:postgres@localhost:5432/ETL Project"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
engine.table_names()

['video_game_scores', 'video_game_sales_by_console']

In [11]:
score_transformed.to_sql(name='video_game_scores', con=engine, if_exists='append', index=True)

In [12]:
sales_transformed.to_sql(name='video_game_sales_by_console', con=engine, if_exists='append', index=True)